## ETL Homework part 1: 
### build a full stack web scraping ETL system
    

We are a web scraping consulting company trying to understand searchrank on Google for a number of clients. We want to build a data asset that tracks a number of keywords top search results.

In [1]:
#1:Build a mongodb collection that maintains a search terms you want to scrape
#2:Build a mongodb collection (initially empty) that will store HTML data
#3:Build a mongodb collection (initially empty) that will store search data
#4:Build a function that makes requests to google that collects the
#   results for the search terms listed in 1) and put them into HTML collection
#5:Call the function from 4) to put HTML data into your HTML collection
#6:Build a function that queries into the HTML data collection, finds any
#   new information, and parses out some information you are interested in
#   (you can be creative here. For instance, what were the top html links. 
#   You could also collect what web domains (github.com, ebay.com) show up)
#   Store the output of this function into the "search data" collection
#7:call the function from 6)

In [2]:
import pymongo 
from splinter import Browser 
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pdb

In [3]:
def web_driver(url):    # this is to visit the url as step 1 
    browser = Browser('chrome',wait_time=10)
    browser.visit(url)
    html= browser.html
    return html

In [4]:
def mongo_connect(dbname):   # make a connection to the mongo db 
    conn= 'mongodb://localhost:27017'
    client= pymongo.MongoClient(conn)
    db= client[dbname]
    return db 

In [5]:
def make_collection(dbname, collections):  # create a collection 
    db = mongo_connect(dbname)
    collections= {c:db[c] for c in collections}
    return db, collections
    

In [6]:
def insert_searchterms(dname,arr):  # a collection specifically for search terms  
    db= mongo_connect(dname)
    db.search_terms.insert_many([{'searchterm':name} for name in arr])
    current_search= list(set([term['searchterm'] for term in db.search_terms.find()]))
    return current_search

In [7]:
def google_scrape_terms(dbname,terms, collections):   # scrape google and dump into html collection
    db = mongo_connect(dbname)
    for st in terms:
        time=datetime.now()
        url= 'https://www.google.com/search?safe=strict&sxsrf=ACYBGNRNfU-ARCBO1sM0NaI5Qs5seb8ghg%3A1571936418813&source=hp&ei=otixXeG8LtWCjLsPg5SV2A0&q='+st+'&oq='+st+'&gs_l=psy-ab.3..35i39j0l3j0i131j0l5.980.1495..1648...0.0..0.127.497.0j4......0....1..gws-wiz.m5c-W7Vvvzw&ved=0ahUKEwjhlea-r7XlAhVVAWMBHQNKBdsQ4dUDCAg&uact=5'
        html= web_driver(url)
        soup= bs(html,'html.parser')
        thehtml=collections.insert_one({'Search term':st,'html':html,'time':time})
    return thehtml, soup

In [8]:
def read_htmlcol(tag,class_,clean):
    raw=[]
    cleaned=[]
    for i in range(len(list(collections['rawhtml'].find()))):
        soup= bs(list(collections['rawhtml'].find())[i]['html'])
        if clean == False:
            soupobject= soup(tag,class_)
            raw.append(soupobject)

        else: 
            soupobject= soup(tag,class_)
            name= [t.text for t in soupobject]
            cleaned.append(name)
    return raw , cleaned

In [9]:
def insert_intocol(current_search,collections,name):
    for st in current_search:  
        time=datetime.now()
        collections.insert_many([{'Search term':st,'time':time,'title':t} for t in name])
    return 

In [10]:
googledb = mongo_connect('googledb')

In [11]:
googledb, collections = make_collection('googledb',['search_terms','rawhtml','websites'])

In [12]:
current_searchterms= insert_searchterms('googledb',['used cars','samsung s10','iphone xr','umberlla'])

In [13]:
thehtml, soup = google_scrape_terms('googledb',current_searchterms, collections['rawhtml'])

In [14]:
websites= read_htmlcol(tag='div',class_='TbwUpd',clean = True)

In [15]:
googleratings=  read_htmlcol(tag='div', class_='DYo30e', clean= True)

In [16]:
insert_intocol(current_searchterms,collections['websites'],websites)

In [32]:
current_searchterms

['used cars', 'samsung s10', 'umberlla', 'iphone xr']

In [33]:
websites

([],
 [['https://www.youtube.com › watch',
   'https://www.amazon.com › umbrella › k=umbrella',
   'nymag.com › strategist › article › best-umbrellas-compact-stick-golf',
   'https://www.target.com › umbrellas-women-s-accessories',
   'https://www.businessinsider.com › best-umbrella',
   'https://en.m.wikipedia.org › wiki › Umbrella',
   'https://m.timesofindia.com › articleshow',
   'https://www.digitaltrends.com › dtdeals › best-windproof-umbrellas',
   'https://www.totes.com › umbrellas',
   'https://en.wikipedia.org › wiki › Umbrella',
   'https://en.wikipedia.org › wiki › Umbrella_(song)',
   'https://www.walmart.com › browse › clothing › umbrellas'],
  ['https://www.samsung.com › mobile › galaxy-s10',
   'https://www.samsung.com › global › galaxy › galaxy-s10',
   'https://www.verizonwireless.com › smartphones › samsung-galaxy-s10',
   'https://www.techradar.com › reviews › samsung-galaxy-s10',
   'https://www.theguardian.com › technology › jun › samsung-galaxy-s10-revi...',
   '